In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import streamlit as st
import io
import os
import sys
import json
import pickle
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# Add modules to path
sys.path.append('modules')

from data_utils import load_processed_data
from model_utils import CNNBaseline

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [3]:
# Load preprocessed data
train_df, val_df, test_df, label_encoder, config = load_processed_data()
print(f"Data loaded: {len(train_df)} train, {len(val_df)} val, {len(test_df)} test samples")

cnn_results = {}
cnn_results['best_model'] = 'resnet50_baseline_best.pth'
cnn_results['best_accuracy'] = 0.5543


with open('models/multimodal_results.json', 'r') as f:
    multimodal_results = json.load(f)
print(f"Multimodal results loaded")
print(f"Multimodal accuracy: {multimodal_results['test_accuracy']:.4f}")
print(f"Improvement: {multimodal_results['improvement']:.4f}")



tfidf_embeddings = np.load('embeddings/tfidf_embeddings.npy')
print(f"TF-IDF embeddings loaded: {tfidf_embeddings.shape}")
    
with open('embeddings/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)
print("TF-IDF vectorizer loaded")
    

Data loaded: 1378 train, 460 val, 460 test samples
Multimodal results loaded
Multimodal accuracy: 0.5957
Improvement: 0.0414
TF-IDF embeddings loaded: (2298, 128)
TF-IDF vectorizer loaded


In [ ]:
# Grad-CAM Implementation for Model Interpretation

class GradCAM:
    """Grad-CAM implementation for CNN model interpretation"""
    
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        self.activations = output.detach()
    
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate_cam(self, input_tensor, class_idx=None):
        """Generate Grad-CAM for given input"""
        # Forward pass
        output = self.model(input_tensor)
        
        if class_idx is None:
            class_idx = output.argmax(dim=1)
        
        # Backward pass
        self.model.zero_grad()
        output[0, class_idx].backward()
        
        # Generate CAM
        gradients = self.gradients[0]  # [C, H, W]
        activations = self.activations[0]  # [C, H, W]
        
        # Global average pooling of gradients
        weights = gradients.mean(dim=(1, 2))  # [C]
        
        # Weighted combination of activation maps
        cam = torch.zeros(activations.shape[1:], dtype=torch.float32)
        for i, w in enumerate(weights):
            cam += w * activations[i]
        
        # Apply ReLU
        cam = F.relu(cam)
        
        # Normalize
        cam = cam - cam.min()
        cam = cam / cam.max()
        
        return cam.detach().numpy()

def create_gradcam_visualization(model, image, class_names, target_layer_name='layer4'):
    """Create Grad-CAM visualization for a given image"""
    
    # Find target layer
    target_layer = None
    for name, module in model.named_modules():
        if name == target_layer_name:
            target_layer = module
            break
    
    if target_layer is None:
        print(f"Target layer {target_layer_name} not found!")
        return None
    
    # Create GradCAM
    gradcam = GradCAM(model, target_layer)
    
    # Generate CAM
    model.eval()
    with torch.no_grad():
        cam = gradcam.generate_cam(image.unsqueeze(0))
    
    return cam

print("Grad-CAM implementation ready!")


Grad-CAM implementation ready!
